In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import bs4
import ssl
import urllib3

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [194]:
# url= "https://www.smartfn.co.kr/article/view/sfn202412230026"
# # url= ["https://www.smartfn.co.kr/article/view/sfn202412230026","https://www.kipost.net/news/articleView.html?idxno=321448"]
# loader = WebBaseLoader(url)
# loader.requests_kwargs = {"verify": False}
# docs = loader.load()
# print(len(docs[0].page_content))

import pandas as pd
from langchain.docstore.document import Document

file_path = "test2.tsv"
df = pd.read_csv(file_path, sep='\t', header=None, names=["ko","en"], skiprows=1)
docs = [Document(page_content=f"한글: ({row['ko']}) -> 영어 번역: ({row['en']})") for _, row in df.iterrows()]


In [195]:
print(len(docs))

7


In [196]:
print(docs[0].page_content)

한글: (가가가) -> 영어 번역: (aaa)


In [197]:
docs

[Document(metadata={}, page_content='한글: (가가가) -> 영어 번역: (aaa)'),
 Document(metadata={}, page_content='한글: (나나나) -> 영어 번역: (bbb)'),
 Document(metadata={}, page_content='한글: (다다다) -> 영어 번역: (ccc)'),
 Document(metadata={}, page_content='한글: (라라라) -> 영어 번역: (ddd)'),
 Document(metadata={}, page_content='한글: (마마마) -> 영어 번역: (eee)'),
 Document(metadata={}, page_content='한글: (사각사각) -> 영어 번역: (sssss)'),
 Document(metadata={}, page_content='한글: (뿌뿌) -> 영어 번역: (bubu)')]

In [198]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len, is_separator_regex=False)
# splits = text_splitter.split_documents(docs)
# print(len(splits))

In [199]:
# embeddings = OllamaEmbeddings(model='llama3:70b',temperature=0)
# vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [200]:
embeddings = OllamaEmbeddings(model='llama3:70b',temperature=0)
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

In [201]:
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k':6})

In [202]:
print(retriever)

tags=['FAISS', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f5bc41f0dc0> search_kwargs={'k': 6}


In [230]:
template = "용어집을 반영해서 한국어를 영어로 번역해줘. \
첫 번째 번역은 <translate_start1>으로 시작하고 <translate_end1>로 끝내줘. \
그 다음 틀린 부분이 있나 검토하고 두 번 째 번역은 <translate_start2>로 시작하고 <translate_end2>로 끝내줘. \
마지막으로 번역한 문장을 다시 한국어로 번역한 후에, 주어진 한국어와 일치하는지 검토한 후, 두 번 째 번역문에서 틀린 부분이 있나 검토하고 마지막 번역은 <translate_start3>로 시작하고 <translate_end3>로 끝내줘. \
용어집 : {context}\n\n한국어 : {question}"
input_prompt = ChatPromptTemplate.from_template(template)

In [231]:
llm = ChatOllama(model='llama3:70b', temperature=0)

In [232]:
output_parser = StrOutputParser()

In [233]:
def merge_info(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [234]:
# chain_level_2 = {'question' : RunnablePassthrough(), 'context' : retriever | merge_info} | input_prompt | llm | output_parser

In [235]:
# print(chain_level_2.invoke({'question' : '현대차 2024년도 영업 실적에 대해서 50자 이내로 말해봐.'}))

In [236]:
chain_level_3 = {'question' : RunnablePassthrough(), 'context':retriever|merge_info} | input_prompt | llm | output_parser

In [237]:
print(chain_level_3.invoke({'question' : '가가가는 나나나인가 다다다인가 그것이 궁금하다. 사각사각 소리가 난다'}))

Let's go through the translation process.

**First Translation**
<translate_start1>
Is aaa going bbb or ccc? That's what I'm curious about. Ssssss sounds are heard.
<translate_end1>

**Review and Correction**
Upon reviewing the first translation,<|begin_of_text|>2019, I noticed that the sentence structure is not natural in English. Also, the word order needs to be adjusted.

**Second Translation**
<translate_start2>
I wonder if it's aaa or bbb that's going ccc. That's what I'm curious about. Meanwhile, I hear ssssss sounds.
<translate_end2>

**Back-Translation and Review**
Let's translate the second translation back into Korean to ensure it matches the original sentence:

한국어: 가가가는 나나나인가 다다다인가 그것이 궁금하다. 사각사각 소리가 난다.

The back-translation looks good!

**Final Translation**
<translate_start3>
I wonder if it's aaa or bbb that's going ccc. That's what I'm curious about. Meanwhile, I hear ssssss sounds.
<translate_end3>

This final translation should accurately convey the meaning of the ori

In [238]:
context = retriever.get_relevant_documents('가가가는 나나나인가 다다다인가 그것이 궁금하다., 사각사각 소리가 난다')
print(context)

[Document(id='2fafb444-7179-493c-85b0-8e992fd27c36', metadata={}, page_content='한글: (라라라) -> 영어 번역: (ddd)'), Document(id='a6897ccf-3a6d-4619-9c93-d6a23e3e3b65', metadata={}, page_content='한글: (사각사각) -> 영어 번역: (sssss)'), Document(id='11c5dd9d-3105-40d8-85fa-1c9586f8b6f3', metadata={}, page_content='한글: (마마마) -> 영어 번역: (eee)'), Document(id='ff8c50f8-b26f-427b-8bd4-19e727c1c0f3', metadata={}, page_content='한글: (가가가) -> 영어 번역: (aaa)'), Document(id='f7a6e69b-d255-4397-8d96-f87d2602dba3', metadata={}, page_content='한글: (나나나) -> 영어 번역: (bbb)'), Document(id='4944583d-127f-4731-a912-2ebdca41bf54', metadata={}, page_content='한글: (다다다) -> 영어 번역: (ccc)')]
